In [2]:
# train_bilstm.py

import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
import re
import html
from tqdm import tqdm
from collections import Counter
from itertools import chain
import numpy as np

# ----------------------------
# 1️⃣ Load & clean dataset
# ----------------------------
df = pd.read_csv(r"R:\AIniverse\PROJECTS\mainstream\senti\data\train.csv\train.csv")  # path to your CSV

label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

def clean_text(text):
    text = str(text)
    text = html.unescape(text)
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"\S+@\S+", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df['comment_text'] = df['comment_text'].apply(clean_text)

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# ----------------------------
# 2️⃣ Build vocabulary
# ----------------------------
all_text = list(train_df['comment_text'])
counter = Counter(chain.from_iterable([x.split() for x in all_text]))
vocab = {w:i+2 for i,(w,c) in enumerate(counter.most_common())}  # +2 for PAD=0, UNK=1
vocab['<PAD>'] = 0
vocab['<UNK>'] = 1
vocab_size = len(vocab)

def encode_text(text, max_len=128):
    tokens = [vocab.get(w,1) for w in text.split()]
    if len(tokens) < max_len:
        tokens += [0]*(max_len-len(tokens))
    else:
        tokens = tokens[:max_len]
    return tokens

# ----------------------------
# 3️⃣ Dataset class
# ----------------------------
class ToxicDataset(Dataset):
    def __init__(self, df):
        self.texts = [encode_text(x) for x in df['comment_text']]
        self.labels = torch.tensor(df[label_cols].values, dtype=torch.float32)
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.texts[idx], dtype=torch.long),
            'labels': self.labels[idx]
        }

train_dataset = ToxicDataset(train_df)
val_dataset = ToxicDataset(val_df)
test_dataset = ToxicDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# ----------------------------
# 4️⃣ BiLSTM + Attention Model
# ----------------------------
class AttentionBiLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=128, output_dim=6):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.attn = nn.Linear(hidden_dim*2, 1)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim*2, output_dim)

    def forward(self, input_ids):
        x = self.embedding(input_ids)
        lstm_out, _ = self.lstm(x)
        attn_weights = torch.softmax(self.attn(lstm_out).squeeze(-1), dim=1).unsqueeze(-1)
        weighted = lstm_out * attn_weights
        pooled = weighted.sum(1)
        out = self.dropout(pooled)
        return self.fc(out)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AttentionBiLSTM(vocab_size).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=2e-3)

In [3]:

# ----------------------------
# 5️⃣ Training with Early Stopping
# ----------------------------
epochs = 20
patience = 3
best_val_loss = float('inf')
early_stop_counter = 0

for epoch in range(epochs):
    model.train()
    train_loss = 0
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    avg_train_loss = train_loss/len(train_loader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    avg_val_loss = val_loss/len(val_loader)
    print(f"Epoch {epoch+1}: Train Loss {avg_train_loss:.4f}, Val Loss {avg_val_loss:.4f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        early_stop_counter = 0
        torch.save(model.state_dict(), "bilstm_toxic_model.pt")
        print("✅ Model improved. Saved.")
    else:
        early_stop_counter +=1
        if early_stop_counter>=patience:
            print("⛔ Early stopping triggered.")
            break

100%|██████████| 3990/3990 [01:17<00:00, 51.16it/s, loss=0.123]  


Epoch 1: Train Loss 0.0618, Val Loss 0.0487
✅ Model improved. Saved.


100%|██████████| 3990/3990 [01:16<00:00, 51.82it/s, loss=0.00469]


Epoch 2: Train Loss 0.0412, Val Loss 0.0474
✅ Model improved. Saved.


100%|██████████| 3990/3990 [01:17<00:00, 51.79it/s, loss=0.000193]


Epoch 3: Train Loss 0.0328, Val Loss 0.0511


100%|██████████| 3990/3990 [01:20<00:00, 49.42it/s, loss=3.13e-5] 


Epoch 4: Train Loss 0.0262, Val Loss 0.0555


100%|██████████| 3990/3990 [01:23<00:00, 47.75it/s, loss=0.0355]  


Epoch 5: Train Loss 0.0213, Val Loss 0.0587
⛔ Early stopping triggered.


In [4]:

# ----------------------------
# 6️⃣ Evaluation
# ----------------------------
model.load_state_dict(torch.load("bilstm_toxic_model.pt"))
model.eval()
all_labels = []
all_preds = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].cpu().numpy()
        outputs = torch.sigmoid(model(input_ids)).cpu().numpy()
        preds = (outputs>0.5).astype(int)
        all_labels.append(labels)
        all_preds.append(preds)

all_labels = np.vstack(all_labels)
all_preds = np.vstack(all_preds)

for i,col in enumerate(label_cols):
    print(f"--- {col} ---")
    print("Accuracy: ", (all_labels[:,i]==all_preds[:,i]).mean())
    print("F1-score:", f1_score(all_labels[:,i], all_preds[:,i]))
    print("ROC-AUC:", roc_auc_score(all_labels[:,i], all_preds[:,i]))

C:\Users\anmol\AppData\Local\Temp\ipykernel_9444\1187294176.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("bilstm_toxic_model.pt"))


--- toxic ---
Accuracy:  0.9634039353302419
F1-score: 0.7956613016095171
ROC-AUC: 0.8670523600002916
--- severe_toxic ---
Accuracy:  0.9899110164180975
F1-score: 0.34285714285714286
ROC-AUC: 0.6283318327190194
--- obscene ---
Accuracy:  0.9798220328361951
F1-score: 0.8031784841075794
ROC-AUC: 0.8796893739394547
--- threat ---
Accuracy:  0.9978067426995865
F1-score: 0.2553191489361702
ROC-AUC: 0.5809554608310968
--- insult ---
Accuracy:  0.974746208798095
F1-score: 0.7401676337846551
ROC-AUC: 0.8496204620462046
--- identity_hate ---
Accuracy:  0.991978944729916
F1-score: 0.36633663366336633
ROC-AUC: 0.6332046208248594


In [ ]:
# bilstm_user_test.py

import torch
import torch.nn as nn
import re
import html
import numpy as np

vocab = torch.load("bilstm_vocab.pt")  # saved vocab
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
max_len = 128

def clean_text(text):
    text = str(text)
    text = html.unescape(text)
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"\S+@\S+", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def encode_text(text):
    tokens = [vocab.get(w,1) for w in text.split()]
    if len(tokens)<max_len:
        tokens += [0]*(max_len-len(tokens))
    else:
        tokens = tokens[:max_len]
    return torch.tensor(tokens).unsqueeze(0)

class BiLSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=128, output_dim=6):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.attention = nn.Linear(hidden_dim*2,1)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.embedding(x)
        out,_ = self.lstm(x)
        attn_weights = torch.softmax(self.attention(out), dim=1)
        out = torch.sum(out*attn_weights, dim=1)
        out = self.dropout(out)
        return self.fc(out)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BiLSTMClassifier(vocab_size=len(vocab), output_dim=len(label_cols))
model.load_state_dict(torch.load("bilstm_toxic_model.pt", map_location=device))
model.eval()
model.to(device)

def predict_toxicity(comment):
    comment = clean_text(comment)
    input_ids = encode_text(comment).to(device)
    with torch.no_grad():
        logits = model(input_ids)
        probs = torch.sigmoid(logits).cpu().numpy()[0]
    result = {label_cols[i]: {'prob': float(probs[i]), 'pred': int(probs[i]>0.5)} for i in range(len(label_cols))}
    return result

while True:
    text = input("Enter comment (or 'quit' to exit): ")
    if text.lower() == 'quit':
        break
    output = predict_toxicity(text)
    print(output)
